# Qtools tutorial 

## Enter Metadata and handle metadata

### Situation

We got a new device, we want to do a measurement on. These are the steps to enter the metadata

1. Add the device to the Metadatabase
2. Create metadata for the measurement
3. Setup the measurement
4. Run the measurement

**Remarks:**

- Steps 1. and 2. can also be done in one step

- During steps 3 and 4, more metadata is gathered with qtools.

### Add device to Metadatabase

Here, we create new metadata for the device, incorporate existing layouts, factories, etc. and save the metadata to the DB.

The helper function guides us through the creation of the metadata.

In [ ]:
from qtools_metadata.metadata import create_metadata_object
from qtools_metadata.device import Device

# Pass Device as argument, to create only the device
device = create_metadata_object(Device)

In [ ]:
device

In [ ]:
from qtools_metadata.metadata import save_metadata_object_to_db
save_metadata_object_to_db(device)
print(device.pid)

### Create metadata for the measurement

To create a complete Metadata set for a measurement, the same helper function is used.

In [ ]:
from qtools_metadata.metadata import create_metadata_object, Metadata

# create_metadata_object() is synonymous
metadata = create_metadata_object(Metadata)

In [ ]:
metadata

In [ ]:
# save metadata to YAML file for future use
with open("metadata.yaml", "w") as f:
    metadata.to_yaml(f)

In [ ]:
metadata.measurement.name = "mynewmeasurement"

In [ ]:
from qtools_metadata.metadata import Metadata

# Load metadata template from YAML file
with open("metadata.yaml", "r") as f:
    metadata: Metadata = Metadata.from_yaml(f)

### Setup the measurement

In [ ]:
import yaml
from qcodes.station import Station
from qcodes_contrib_drivers.drivers.QDevil.QDAC1 import QDac
from qtools.instrument.custom_drivers.ZI.MFLI import MFLI
from qtools.instrument.mapping import (
    
    
    add_mapping_to_instrument,
    MFLI_MAPPING,
    QDAC_MAPPING)
from qtools.instrument.mapping.base import map_gates_to_instruments
from qtools.measurement.scripts import (
    Generic_1D_Sweep,
    Generic_nD_Sweep,
    Generic_1D_parallel_Sweep,
    Timetrace,
    Timetrace_with_sweeps)
from qtools.utils.load_from_sqlite_db import load_db
from qtools.utils.generate_sweeps import generate_sweep, replace_parameter_settings, update_parameter_settings
from qtools.utils.ramp_parameter import *
from qcodes.dataset import load_or_create_experiment

In [ ]:
# Setup qcodes station
station = Station()

qdac = QDac("qdac", "ASRL5::INSTR")
add_mapping_to_instrument(qdac, QDAC_MAPPING)
station.add_component(qdac)

lockin = MFLI("lockin", "DEV4039")#, serverhost = "192.168.1.142")
add_mapping_to_instrument(lockin, MFLI_MAPPING)
station.add_component(lockin)


In [ ]:
load_db()


In [ ]:
load_or_create_experiment(experiment_name = metadata.measurement.series.name,
                          sample_name = f"{metadata.measurement.device.sample.name}_{metadata.measurement.device.name}")

In [ ]:
with open("C:/Users/lab/Documents/Measurements/Huckemann/Settings/QBus.yaml", "r") as file:
    parameters = yaml.safe_load(file)
replace_parameter_settings(parameters, "setpoints", generate_sweep(0.8, 1.5, 150))
parameters

In [ ]:
script1 = Generic_1D_parallel_Sweep()
script1.setup(parameters, metadata, backsweep_after_break = True, ramp_time = 0.5)

In [ ]:
map_gates_to_instruments(station.components, script1.gate_parameters)

**Save mapping for future use...**

In [ ]:
mapping = script1.gate_parameters

### Run the measurement

Metadata is saved automatically to the DB.

In [ ]:
from qtools.utils.GUI import *
open_web_gui(script1)

In [ ]:
script1.run()

### Access Measurements metadata

In [ ]:
from pprint import pprint
from qtools_metadata.measurement import Measurement
measurements = [meas for meas in Measurement.get_all() if meas.device.sample.name == "12-2"]
pprint(measurements)

### Load metadata from YAML

In [ ]:
from pprint import pprint
from qtools_metadata.metadata import Metadata

# Load metadata template from YAML file
with open("metadata.yaml", "r") as f:
    metadata: Metadata = Metadata.from_yaml(f)

# Change some stuff
metadata.measurement.name = "Gate Pinchoffs"
#metadata.measurement.comments = "a second measurement"

# Next measurement...

### 2nd Measurement

In [ ]:
with open("C:/Users/lab/Documents/Measurements/Huckemann/Settings/QuBus_pinchoffs.yaml", "r") as file:
    parameters = yaml.safe_load(file)
replace_parameter_settings(parameters, "setpoints", generate_sweep(1.18, 0, 200, include_backward=True))

In [ ]:
script2 = Generic_1D_Sweep()
script2.setup(parameters, metadata, ramp_time = 0.5)

In [ ]:
map_gates_to_instruments(station.components, script2.gate_parameters, map_manually=True)

In [ ]:
script2.run()